<a href="https://colab.research.google.com/github/sorielkobayashi/Custom-Spleeter-Colab/blob/main/Neural%20Audio%20Stem%20Separation%20(Espa%C3%B1ol).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Spleeter Colab

Esta es una extensión para [Deezer Spleeter](https://github.com/deezer/spleeter) que se ejecuta en Google Colab, utilizando Google Drive como almacenamiento de datos.

El valor de `input` puede ser
1.   Ruta a un archivo de audio individual ubicado en su Google Drive.
2.   Ruta a un directorio en su Google Drive que contenga varios archivos de audio. Cada archivo de audio encontrado en el directorio será procesado individualmente.
3.   URL individual de YouTube
4.   Múltiples URLs de YouTube separadas por espacios. Cada vídeo de YouTube será procesado individualmente.

La entrada será separada en 4 stems: voz, batería, bajo y otros. Los stems se guardarán como archivos WAV en su Google Drive. Introduzca `output_dir` relativo a su raíz de Google Drive.

In [ ]:
#@title Setup
#@markdown This cell needs to be run only once. It will mount your Google Drive and setup prerequisites.

import os
from google.colab import drive, output
drive.mount('/content/drive')
drive_root = '/content/drive/MyDrive/'

# inhagcutils
!pip -q install import-ipynb
!curl -s -O https://raw.githubusercontent.com/olaviinha/inhagcutils/master/inhagcutils.ipynb
import import_ipynb
from inhagcutils import *
!gsutil -q -m cp -R gs://neural-research/olaviinha/spleeter-configs/custom-4stems-22kHz-z.json /content/cfg.json

#@title Install
!apt -q install ffmpeg
!pip -q install spleeter

output.clear()
print('Using config gs://neural-research/olaviinha/spleeter-configs/custom-4stems-22kHz-z.json')
op(c.ok, 'Setup finished.')


In [ ]:
#@title Separate

input = "/content/01. El Eco de Tu Adios.flac" #@param {type:"string"}
output_dir = "/stems3" #@param {type:"string"}

#----------------------------------------------------------------------------#

import os
from os.path import isfile, join
from glob import glob

input_type = check_input_type(input)
tmp_dir = '/content/tmp/'
seps_dir = fix_path(drive_root+output_dir)
#seps_dir = '/content/tmp/seps'

if not os.path.isdir(tmp_dir):
  create_dirs([tmp_dir])
# if not os.path.isdir(output_dir):
#   create_dirs([seps_dir])

if output_dir == '' or output_dir == 'same':
  output_dir = fix_path(drive_root+path_dir(input))
else:
  output_dir = fix_path(drive_root+output_dir)
  if not os.path.isdir(output_dir):
    create_dirs([output_dir])

if input_type == 'unknown':
  input = drive_root+input
  input_type = check_input_type(input)
  if input_type=='dir':
    input = fix_path(input)

if input_type == 'youtube':
  !pip -q install youtube-dl
  !youtube-dl --restrict-filenames -x --no-continue --audio-format wav -o "{tmp_dir}%(title)s.%(ext)s" {input}

from spleeter.separator import Separator
separator = Separator('/content/cfg.json')

if input_type == "dir" or input_type == "youtube":
  if input_type == "dir":
    target = input
  if input_type == "youtube":
    target = dir_tmp
  file_list = list_audio(target)
  for audiofile in file_list:
    op(c.title, '\nSeparate '+audiofile+' to '+seps_dir.replace(drive_root, '')+basename(input))
    # Spleeter Python API is outdated for Colab
    # separator.separate_to_file(audiofile, seps_dir)
    !spleeter separate -p /content/cfg.json -o "{output_dir}" "{audiofile}"
    op(c.ok, 'FIN.', 'Files saved under directory '+output_dir.replace(drive_root, '')+basename(audiofile)+'/')
elif input_type == "file":
  # Spleeter Python API is outdated for Colab
  # separator.separate_to_file(input, seps_dir)
  !spleeter separate -p /content/cfg.json -o "{seps_dir}" "{input}"
  op(c.ok, 'FIN.', 'Files saved under directory '+output_dir.replace(drive_root, '')+basename(input)+'/')
else:
  print("We're not sure what you want to separate. Please check the input field.")

In [ ]:
#@title Comprimir Carpeta

input = "/content/drive/MyDrive/stems3/01. El Eco de Tu Adios" #@param {type:"string"}


#----------------------------------------------------------------------------#



from google.colab import files
!apt-get install rar
!rar a "{input}".rar "{input}"



In [ ]:
#@title Separate

input = "/content/08. Gavil\xE1n o Paloma.flac" #@param {type:"string"}
output_dir = "/stems3" #@param {type:"string"}

#----------------------------------------------------------------------------#

import os
from os.path import isfile, join
from glob import glob

input_type = check_input_type(input)
tmp_dir = '/content/tmp/'
seps_dir = fix_path(drive_root+output_dir)
#seps_dir = '/content/tmp/seps'

if not os.path.isdir(tmp_dir):
  create_dirs([tmp_dir])
# if not os.path.isdir(output_dir):
#   create_dirs([seps_dir])

if output_dir == '' or output_dir == 'same':
  output_dir = fix_path(drive_root+path_dir(input))
else:
  output_dir = fix_path(drive_root+output_dir)
  if not os.path.isdir(output_dir):
    create_dirs([output_dir])

if input_type == 'unknown':
  input = drive_root+input
  input_type = check_input_type(input)
  if input_type=='dir':
    input = fix_path(input)

if input_type == 'youtube':
  !pip -q install youtube-dl
  !youtube-dl --restrict-filenames -x --no-continue --audio-format wav -o "{tmp_dir}%(title)s.%(ext)s" {input}

from spleeter.separator import Separator
separator = Separator('/content/cfg.json')

if input_type == "dir" or input_type == "youtube":
  if input_type == "dir":
    target = input
  if input_type == "youtube":
    target = dir_tmp
  file_list = list_audio(target)
  for audiofile in file_list:
    op(c.title, '\nSeparate '+audiofile+' to '+seps_dir.replace(drive_root, '')+basename(input))
    # Spleeter Python API is outdated for Colab
    # separator.separate_to_file(audiofile, seps_dir)
    !spleeter separate -p /content/cfg.json -o "{output_dir}" "{audiofile}"
    op(c.ok, 'FIN.', 'Files saved under directory '+output_dir.replace(drive_root, '')+basename(audiofile)+'/')
elif input_type == "file":
  # Spleeter Python API is outdated for Colab
  # separator.separate_to_file(input, seps_dir)
  !spleeter separate -p /content/cfg.json -o "{seps_dir}" "{input}"
  op(c.ok, 'FIN.', 'Files saved under directory '+output_dir.replace(drive_root, '')+basename(input)+'/')
else:
  print("We're not sure what you want to separate. Please check the input field.")

In [ ]:
#@title Comprimir Carpeta

input = "/content/drive/MyDrive/stems3/08. Gavil\xE1n o Paloma" #@param {type:"string"}


#----------------------------------------------------------------------------#



from google.colab import files
!apt-get install rar
!rar a "{input}".rar "{input}"



In [ ]:
#@title Separate

input = "/content/10. El Amar y El Querer.flac" #@param {type:"string"}
output_dir = "/stems3" #@param {type:"string"}

#----------------------------------------------------------------------------#

import os
from os.path import isfile, join
from glob import glob

input_type = check_input_type(input)
tmp_dir = '/content/tmp/'
seps_dir = fix_path(drive_root+output_dir)
#seps_dir = '/content/tmp/seps'

if not os.path.isdir(tmp_dir):
  create_dirs([tmp_dir])
# if not os.path.isdir(output_dir):
#   create_dirs([seps_dir])

if output_dir == '' or output_dir == 'same':
  output_dir = fix_path(drive_root+path_dir(input))
else:
  output_dir = fix_path(drive_root+output_dir)
  if not os.path.isdir(output_dir):
    create_dirs([output_dir])

if input_type == 'unknown':
  input = drive_root+input
  input_type = check_input_type(input)
  if input_type=='dir':
    input = fix_path(input)

if input_type == 'youtube':
  !pip -q install youtube-dl
  !youtube-dl --restrict-filenames -x --no-continue --audio-format wav -o "{tmp_dir}%(title)s.%(ext)s" {input}

from spleeter.separator import Separator
separator = Separator('/content/cfg.json')

if input_type == "dir" or input_type == "youtube":
  if input_type == "dir":
    target = input
  if input_type == "youtube":
    target = dir_tmp
  file_list = list_audio(target)
  for audiofile in file_list:
    op(c.title, '\nSeparate '+audiofile+' to '+seps_dir.replace(drive_root, '')+basename(input))
    # Spleeter Python API is outdated for Colab
    # separator.separate_to_file(audiofile, seps_dir)
    !spleeter separate -p /content/cfg.json -o "{output_dir}" "{audiofile}"
    op(c.ok, 'FIN.', 'Files saved under directory '+output_dir.replace(drive_root, '')+basename(audiofile)+'/')
elif input_type == "file":
  # Spleeter Python API is outdated for Colab
  # separator.separate_to_file(input, seps_dir)
  !spleeter separate -p /content/cfg.json -o "{seps_dir}" "{input}"
  op(c.ok, 'FIN.', 'Files saved under directory '+output_dir.replace(drive_root, '')+basename(input)+'/')
else:
  print("We're not sure what you want to separate. Please check the input field.")

In [ ]:
#@title Comprimir Carpeta

input = "/content/drive/MyDrive/stems3/10. El Amar y El Querer" #@param {type:"string"}


#----------------------------------------------------------------------------#



from google.colab import files
!apt-get install rar
!rar a "{input}".rar "{input}"



In [ ]:
#@title Separate

input = "/content/13. Si Me Dejas Ahora.flac" #@param {type:"string"}
output_dir = "/stems3" #@param {type:"string"}

#----------------------------------------------------------------------------#

import os
from os.path import isfile, join
from glob import glob

input_type = check_input_type(input)
tmp_dir = '/content/tmp/'
seps_dir = fix_path(drive_root+output_dir)
#seps_dir = '/content/tmp/seps'

if not os.path.isdir(tmp_dir):
  create_dirs([tmp_dir])
# if not os.path.isdir(output_dir):
#   create_dirs([seps_dir])

if output_dir == '' or output_dir == 'same':
  output_dir = fix_path(drive_root+path_dir(input))
else:
  output_dir = fix_path(drive_root+output_dir)
  if not os.path.isdir(output_dir):
    create_dirs([output_dir])

if input_type == 'unknown':
  input = drive_root+input
  input_type = check_input_type(input)
  if input_type=='dir':
    input = fix_path(input)

if input_type == 'youtube':
  !pip -q install youtube-dl
  !youtube-dl --restrict-filenames -x --no-continue --audio-format wav -o "{tmp_dir}%(title)s.%(ext)s" {input}

from spleeter.separator import Separator
separator = Separator('/content/cfg.json')

if input_type == "dir" or input_type == "youtube":
  if input_type == "dir":
    target = input
  if input_type == "youtube":
    target = dir_tmp
  file_list = list_audio(target)
  for audiofile in file_list:
    op(c.title, '\nSeparate '+audiofile+' to '+seps_dir.replace(drive_root, '')+basename(input))
    # Spleeter Python API is outdated for Colab
    # separator.separate_to_file(audiofile, seps_dir)
    !spleeter separate -p /content/cfg.json -o "{output_dir}" "{audiofile}"
    op(c.ok, 'FIN.', 'Files saved under directory '+output_dir.replace(drive_root, '')+basename(audiofile)+'/')
elif input_type == "file":
  # Spleeter Python API is outdated for Colab
  # separator.separate_to_file(input, seps_dir)
  !spleeter separate -p /content/cfg.json -o "{seps_dir}" "{input}"
  op(c.ok, 'FIN.', 'Files saved under directory '+output_dir.replace(drive_root, '')+basename(input)+'/')
else:
  print("We're not sure what you want to separate. Please check the input field.")

In [ ]:
#@title Comprimir Carpeta

input = "/content/drive/MyDrive/stems3/13. Si Me Dejas Ahora" #@param {type:"string"}


#----------------------------------------------------------------------------#



from google.colab import files
!apt-get install rar
!rar a "{input}".rar "{input}"

